In [1]:
# !pip install bert-score

In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [8]:
import pandas as pd
import os

In [9]:
# from bert_score import score

In [10]:
df=pd.read_csv("gpt4-100.csv")

In [11]:
df.keys()

Index(['context', 'question', 'references', 'v17', 'v17_gpt4', 'v17_q1',
       'v17_q2', 'v17_q3', 'v17_q4', 'openthaigpt', 'openthaigpt_gpt4',
       'openthaigpt_q1', 'openthaigpt_q2', 'openthaigpt_q3', 'openthaigpt_q4',
       'seallm', 'seallm_gpt4', 'seallm_q1', 'seallm_q2', 'seallm_q3',
       'seallm_q4', 'polylm', 'polylm_gpt4', 'polylm_q1', 'polylm_q2',
       'polylm_q3', 'polylm_q4', 'openthaigpt13b', 'openthaigpt13b_gpt4',
       'openthaigpt13b_q1', 'openthaigpt13b_q2', 'openthaigpt13b_q3',
       'openthaigpt13b_q4', 'typhoon', 'typhoon_gpt4', 'typhoon_q1',
       'typhoon_q2', 'typhoon_q3', 'typhoon_q4'],
      dtype='object')

In [12]:
refs=df.references.tolist()

In [30]:
from pythainlp.tokenize import word_tokenize
import string
import re
from collections import Counter
"""
XQuAD
"""
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def cut_thai(text): # Add!!!
        return ' '.join(word_tokenize(text,engine="newmm"))

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(cut_thai(s)))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def compute_score(references, predictions):
    f1 = exact_match = total = 0
    for _ground_truths,_prediction in list(zip(references, predictions)):
        total += 1
        ground_truths = [_ground_truths]
        prediction = _prediction
        exact_match += metric_max_over_ground_truths(exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {"exact_match": exact_match, "f1": str(f1)}

In [31]:
cands=df.v17.tolist()
v17_xquad=[compute_score([r],[p]) for r,p in zip(refs,cands)]
F1_v17 = [i["f1"] for i in v17_xquad]
cands=df.openthaigpt.tolist()
openthaigpt_xquad=[compute_score([r],[p]) for r,p in zip(refs,cands)]
F1_otg = [i["f1"] for i in openthaigpt_xquad]
cands=df.seallm.tolist()
seallm_xquad=[compute_score([r],[p]) for r,p in zip(refs,cands)]
F1_seallm = [i["f1"] for i in seallm_xquad]
f1_list=F1_v17+F1_otg+F1_seallm
f1_list=[str(i) for i in f1_list]

In [32]:
print('\n'.join(F1_v17))

100.0
66.66666666666666
0.0
100.0
0.0
0.0
100.0
0.0
12.5
0.0
0.0
0.0
0.0
66.66666666666666
0.0
66.66666666666666
100.0
0.0
100.0
71.42857142857143
0.0
100.0
66.66666666666666
100.0
85.71428571428571
66.66666666666666
0.0
74.99999999999999
0.0
66.66666666666666
66.66666666666666
100.0
66.66666666666666
100.0
88.8888888888889
0.0
0.0
66.66666666666666
0.0
0.0
0.0
100.0
0.0
85.71428571428571
0.0
66.66666666666666
100.0
100.0
100.0
44.44444444444445
7.6923076923076925
100.0
100.0
0.0
100.0
100.0
100.0
100.0
100.0
66.66666666666666
0.0
100.0
0.0
100.0
0.0
0.0
12.5
100.0
0.0
33.333333333333336
0.0
0.0
0.0
0.0
100.0
66.66666666666666
0.0
0.0
100.0
100.0
66.66666666666666
100.0
40.0
100.0
36.36363636363636
66.66666666666666
100.0
100.0
18.181818181818183
4.651162790697675
0.0
0.0
66.66666666666666
100.0
88.8888888888889
100.0
66.66666666666666
0.0
40.0
0.0


In [33]:
print('\n'.join(F1_otg))

25.0
15.384615384615385
0.0
66.66666666666666
0.0
0.0
0.0
0.0
0.0
100.0
0.0
0.0
0.0
2.9850746268656723
11.11111111111111
11.11111111111111
0.0
0.0
15.384615384615385
55.55555555555556
0.0
0.0
0.0
30.76923076923077
0.0
66.66666666666666
26.66666666666667
20.0
25.0
0.0
13.333333333333334
100.0
13.333333333333334
71.42857142857143
38.095238095238095
0.0
100.0
15.384615384615385
100.0
66.66666666666666
0.0
100.0
0.0
18.75
100.0
40.0
100.0
0.0
50.0
16.666666666666664
16.666666666666668
0.0
100.0
72.72727272727272
100.0
100.0
100.0
0.0
100.0
25.0
0.0
0.0
0.0
100.0
0.0
0.0
0.0
0.0
25.0
22.22222222222222
0.0
0.0
0.0
100.0
0.0
0.0
40.0
0.0
0.0
33.33333333333333
100.0
66.66666666666666
100.0
44.44444444444445
0.0
0.0
100.0
100.0
11.76470588235294
100.0
66.66666666666666
18.181818181818183
66.66666666666666
0.0
30.769230769230766
100.0
33.33333333333333
66.66666666666666
40.0
100.0


In [34]:
print('\n'.join(F1_seallm))

0.0
11.76470588235294
18.181818181818183
0.0
0.0
0.0
0.0
0.0
0.0
6.8181818181818175
13.333333333333334
0.0
0.0
0.0
26.08695652173913
10.526315789473683
0.0
10.81081081081081
0.0
30.303030303030305
15.384615384615385
0.0
4.255319148936169
21.052631578947366
19.354838709677416
10.526315789473683
17.391304347826082
27.27272727272727
17.391304347826082
5.555555555555556
8.695652173913045
14.634146341463413
8.333333333333332
33.333333333333336
42.10526315789474
19.04761904761905
46.15384615384615
9.523809523809524
0.0
6.89655172413793
11.11111111111111
23.52941176470588
0.0
47.05882352941176
32.0
12.5
30.0
0.0
4.444444444444445
0.0
14.285714285714288
31.57894736842105
17.391304347826082
0.0
0.0
38.46153846153846
42.10526315789474
0.0
14.084507042253522
13.333333333333334
0.0
24.24242424242424
0.0
0.0
19.04761904761905
10.81081081081081
4.651162790697675
10.0
12.5
10.526315789473683
18.181818181818183
16.666666666666668
0.0
8.695652173913045
25.806451612903224
50.0
15.384615384615385
15.7894

In [35]:
cands=df.v17.tolist()
print("v17")
print(compute_score(refs,cands))
cands=df.openthaigpt.tolist()
print("openthaigpt")
print(compute_score(refs,cands))
cands=df.seallm.tolist()
print("seallm")
print(compute_score(refs,cands))

v17
{'exact_match': 32.0, 'f1': '50.11968290107823'}
openthaigpt
{'exact_match': 21.0, 'f1': '34.960694330788705'}
seallm
{'exact_match': 0.0, 'f1': '14.361506623099723'}
